### **Preprocess PDF using TESSERACT OCR**

In [1]:
import cv2
import numpy as np
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
import re
import os
from paddleocr import PaddleOCR
from utils import (is_cn_block,
                   #ocr_layout,
                   extract_vn_letters,
                   extract_cn_letters,)

d:\DOCUMENT\School\NLP\school\Project\mt-project\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
%load_ext autoreload
%autoreload 2

In [2]:
current_dir = os.getcwd()
#poppler_path_windows = r"D:\DOCUMENT\School\NLP\school\Project\poppler-25.11.0\Library\bin"
poppler_path_windows = os.path.join(current_dir, 'bin', 'poppler-25.11.0', 'Library', 'bin')
#path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
path_to_tesseract = os.path.join(current_dir, 'bin', 'Tesseract-OCR', 'tesseract.exe')
pytesseract.pytesseract.tesseract_cmd = path_to_tesseract

start_page = 107
end_page = 157

In [3]:
pages = convert_from_path('data/raw/PDF1_999 buc thu viet cho ban than.pdf', 
                          poppler_path=poppler_path_windows, 
                          dpi=400,
                          first_page = start_page,
                          last_page = 127,
                          )

In [27]:
paddle_ocr = PaddleOCR(lang='ch', 
                use_doc_orientation_classify=False,
                use_doc_unwarping=False,
                use_textline_orientation=False,)

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\vnviv\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\vnviv\.paddlex\official_models\PP-OCRv5_server_rec`.


In [28]:
def ocr_layout(pil_img, paddle_ocr, index):
    cn_text = ""
    vn_text = ""
    
    img = np.array(pil_img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Upscale 2 lần  
    #scaled = cv2.resize(img, None, fx=2, fy=2, interpolation= cv2.INTER_CUBIC)  # INTER_CUBIC giữ nét khi phóng to
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Tạo khối
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,10))
    dilated = cv2.dilate(thresh, kernel, iterations = 1)
    
    # Tìm contours (các khung bao quanh văn bản)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    blocks = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if w > 50 and h > 10:
            blocks.append((x,y,w,h))
            
    # Sort lại theo trục y
    blocks.sort(key = lambda x : x[1])
    
    final_result = ""
    
    # Duyệt từng block và xử lý theo tiếng Trung - Viêt
    
    if not os.path.exists('debug_images'):
        os.makedirs('debug_images')
        
    h_img, w_img = img.shape[:2]
    
    for i, (x,y,w,h) in enumerate(blocks):
        margin = 10
        y_min, y_max = max(0, y-(margin+5)), min(h_img, y+h+margin+5)
        x_min, x_max = max(0, x-margin), min(w_img, x+w+margin)
        
        roi = img[y_min:y_max, x_min:x_max]
        
        if len(roi.shape) == 2:  # Nếu là ảnh xám (chỉ có Height, Width)
            roi = cv2.cvtColor(roi, cv2.COLOR_GRAY2BGR)
            
        if roi.shape[0] < 40: 
            roi = cv2.resize(roi, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
            
        #text_chi = pytesseract.image_to_string(roi, lang='chi_sim', config='--psm 6')
        
        roi_padded = cv2.copyMakeBorder(roi, 20, 20, 20, 20, cv2.BORDER_CONSTANT, value=[255, 255, 255])
        
        cv2.imwrite(f'debug_images/roi_block_{index}_{i}.jpg', roi_padded)
        
        paddle_result = paddle_ocr.ocr(roi_padded)
        
        text_list = []
        
        for res in paddle_result:
            text_list.append(res['rec_texts'])  
            
        with open("debug_text.txt", "a", encoding = 'utf-8') as f:
            f.write(str(text_list))
            f.write("\n")
            
        text_chi = "".join(text_list[0])
        #print(text_chi)
        
        if is_cn_block(text_chi):
            cn_text += text_chi.strip() + "\n"
        else:
            text_vie = pytesseract.image_to_string(roi, lang='vie', config='--psm 6')
            vn_text += text_vie.strip() + "\n"
            
    return cn_text, vn_text

In [29]:
full_text = {}
full_text['CN'] = []
full_text['VN'] = []

for i, page_image in enumerate(pages):
    cn_text, vn_text = ocr_layout(page_image, paddle_ocr, i)
    full_text['CN'].append(cn_text)
    # vn_text sẽ chứa tiếng việt và tiếng pinyin -> sẽ xử lý sau
    full_text['VN'].append(vn_text)

C:\Users\vnviv\AppData\Local\Temp\ipykernel_18480\2010310957.py:57: DeprecationWarning: Please use `predict` instead.
  paddle_result = paddle_ocr.ocr(roi_padded)


In [ ]:
letters_vn = []
for v in full_text['VN']:
    letters_vn.extend(extract_vn_letters(v))
    
letters_vn

['Bức thư viết cho chính mình số 251 Độc thân cũng tốt, yêu đương cũng được. Tất cả chỉ là một giai đoạn nào đó trong cuộc đời bạn. Không phải hâm mộ những người xung quanh thì cũng đừng bất mãn với hiện tại. Cho dù có ai đó bên cạnh hay không, phía trước có lời hứa cùng phấn đấu vì một mục tiêu nào đó hay không, mọi cung bậc cảm xúc trong cuộc sống đều cần chính bản thân trải nghiệm qua và đúc kết. Sống đúng thì mỗi giai đoạn đều sẽ trở thành một bản ngã tốt hơn của chính bạn. Trưởng thành không chỉ đơn giản là để thoát khỏi cảnh độc thân mà là khiến viết cho chính mình có thế trải nghiệm tất cả những việc gì “có khả năng”.',
 'Bức thư viết cho chính mình số 252 Làm việc khi tính táo, đọc sách khi mơ hồ, ngủ khi tức giận và suy nghĩ khi ở một mình. Hãy làm một người hạnh phúc, đọc sách, du lịch, nỗ lực làm việc, quan tâm đến cơ thể và tâm lí, đồng thời trở thành người tốt nhất so với chính mình.',
 'Bức thư viết cho chính mình số 253 Con người có ba lần đánh dấu sự trưởng thành: Lần đ

In [33]:
len(letters_vn)

54

In [37]:
letters_cn = []

for v in full_text['CN']:
    letters_cn.extend(extract_cn_letters(v))
    
letters_cn

['写给自己的第251封 信 单身也好， 没必要羡慕旁人， 都只是人生中的某个阶段。 恋爱也罢， 无论身边是否有人陪伴， 前方是否有承诺一起努力的 也别对现状抱不满。 正确地活着， 生活中的喜怒哀乐都需要自己去经历并沉淀的。 每个 目标， 阶段都会成就更好的自己。 而是让自己有力量 成长不是单纯地为了脱单， 去体验任何可能。',
 '写给自己的第252封 信 清醒时做事， 大怒时睡觉， 独处时思考； 做一个幸福的 糊涂时读书， 成为最好的自己。 关心身体和心情， 努力工作， 读书， 旅行，',
 '写给自己的第253封 信 第二次是 第一次是在发现自己不是世界中心的时候。 人会长大三次。 第三次是 终究还是有些事令人无能为力的时候。 在发现即使再怎么努力， 但还是会尽力争取的时候。 在明知道有些事可能会无能为力，',
 '写给自己的第254封 信 我学会了勇敢; 我学会了坚强; 怕的时候没人陪， 哭的时候没人哄， 我学会了自 累的时候没人可以依靠， 我学会了承受; 烦的时候没人问， 原来我很优秀， 我 我找到了自己， 更可贵的是， 立……就这样， 世界上， 只有一个我！ 只有一个，',
 '写给自己的第255封 信 可以游手好闲而赢得女人的欣赏。 更没有任何一 没有任何一个男人， 而得到一个男人的尊重。 个女人能够好吃懒做， 别 还是埋头干吧， 所以， 没有委屈就没有成长。 把时间浪费在解释上，',
 '写给自己的第256封 信 你能多快搞定自己的情绪， 就能多快得到成长。脾气会赶走运气。 所 在成为你想要成为的人之前， 不过就是能忍。 那些貌似心大的人， 做 忍着。 活着， 好两件事：',
 '写给自己的第257封 信 会是你明天的成 可以心疼， 你可以哭泣， 今天的泪水， 但不能绝望。 今天的伤痕，会是你明天的坚强。',
 '写给自己的第258封 信 当你失去了所有的依靠 这回事， 这个世界上根本就不存在 自然就什么都会了。 的时候，',
 '写给自己的第259封 信 那些受伤的地方一定会变成我 生活总是让我们遍体鳞伤， 但到后来， 们最强壮的地方。',
 '写给自己的第260封 信 如果真的每个人都是鸿鹄的话， 那么便没有燕雀 成长路上无捷径， “天才是百分之一的天分加上百分之九十九的汗水。 爱迪生说: 这 就必须学会努力和付出以及 想成功， 想成长、 也证明

In [39]:
len(letters_cn)

54

In [40]:
import pandas as pd 

df_vn = pd.DataFrame(letters_vn, columns =['vi'])
df_cn = pd.DataFrame(letters_cn[:-1], columns = ['zh'])

In [41]:
df_com = pd.concat([df_vn, df_cn], axis=1, ignore_index=True)

In [42]:
df_com

,0,1
0,Bức thư viết cho chính mình số 251 Độc thân cũ...,写给自己的第251封 信 单身也好， 没必要羡慕旁人， 都只是人生中的某个阶段。 恋爱也罢，...
1,Bức thư viết cho chính mình số 252 Làm việc kh...,写给自己的第252封 信 清醒时做事， 大怒时睡觉， 独处时思考； 做一个幸福的 糊涂时读书...
2,Bức thư viết cho chính mình số 253 Con người c...,写给自己的第253封 信 第二次是 第一次是在发现自己不是世界中心的时候。 人会长大三次。 ...
3,Bức thư viết cho chính mình số 254 Khi khóc kh...,写给自己的第254封 信 我学会了勇敢; 我学会了坚强; 怕的时候没人陪， 哭的时候没人哄，...
4,Bức thư viết cho chính mình số 255 Không có mộ...,写给自己的第255封 信 可以游手好闲而赢得女人的欣赏。 更没有任何一 没有任何一个男人， ...
5,Bức thư viết cho chính mình số 256 Bạn càng ki...,写给自己的第256封 信 你能多快搞定自己的情绪， 就能多快得到成长。脾气会赶走运气。 所 ...
6,Bức thư viết cho chính mình số 257 Bạn có thể ...,写给自己的第257封 信 会是你明天的成 可以心疼， 你可以哭泣， 今天的泪水， 但不能绝望...
7,Bức thư viết cho chính mình số 258 Trên thế gi...,写给自己的第258封 信 当你失去了所有的依靠 这回事， 这个世界上根本就不存在 自然就什么...
8,Bức thư viết cho chính mình số 259 Cuộc sống l...,写给自己的第259封 信 那些受伤的地方一定会变成我 生活总是让我们遍体鳞伤， 但到后来， ...
9,Bức thư viết cho chính mình số 260 Con đường t...,写给自己的第260封 信 如果真的每个人都是鸿鹄的话， 那么便没有燕雀 成长路上无捷径， “...
